In [3]:
import requests
import json
import os
import hashlib
home_url = 'http://192.168.163.1'
release_path = "C:/DTLDownLoads/"
def newest_file(path, with_path=True):
    file_dict = {}
    lists = os.listdir(path)
    for file_name in lists:
        ctime = os.stat(os.path.join(path, file_name)).st_ctime
        file_dict[ctime] = with_path and os.path.join(path, file_name) or file_name
    max_ctime = max(file_dict.keys())
    return file_dict[max_ctime]

def new_bin():
    return newest_file(release_path)

def login(username='uu', password='UU123456'):
    resp = requests.post(home_url + '/api/sys/login', data={
        "username": username,
        "password": password
    })
    print(resp.headers)
    if resp.status_code == 200:
        js = json.loads(resp.text)
        return js['code'] == 0 and js['message'] == 'success'
    else:
        return False
    
def upgrade(filename, version_code=1001, version_name="uu_v1.0"):
    m = ''
    key = bytes.fromhex('AEA9260668D329452A8B7FF833CA0AFA92EE56A114F9591C71761577A2D671DF')
    md5 = ''
    
    print(len(key), key)
    with open(filename, 'rb') as f:
        m = hashlib.md5() 
        m.update(key)
        m.update(f.read())
        md5 = m.hexdigest()
        print('md5=', md5)
    
    length = os.path.getsize(filename)
    print('length=', length)

    resp = requests.post(home_url + '/api/sys/firmware', data={
        "version_code": 1002,
        "version_name": 'uu_1.1',
        "firmware_size": length,
        "firmware_sign": md5
    }, files = {"file": ("1111.bin", open(filename, 'rb'))})

    print(resp.status_code)
    if resp.status_code == 200:
        js = json.loads(resp.text)
        print(js['message'])
        return js['code'] == 0 and js['message'] == 'success'
    else:
        print(resp.text)
        return False
    
print(login())

{'Date': 'Tue, 13 Nov 2018 15:41:56 GMT', 'Server': 'Boa/0.94.14rc21', 'Accept-Ranges': 'bytes', 'Connection': 'close', 'Content-Length': '30', 'Content-Type': 'application/json; charset=utf-8'}
True


In [4]:
bin_file = newest_file(release_path)
bin_file_size = os.path.getsize(bin_file)
print(bin_file, bin_file_size)
print(upgrade(new_bin()))

C:/DTLDownLoads/111.bin 5713956
32 b'\xae\xa9&\x06h\xd3)E*\x8b\x7f\xf83\xca\n\xfa\x92\xeeV\xa1\x14\xf9Y\x1cqv\x15w\xa2\xd6q\xdf'
md5= b39a5fcfecfd85a8b211cb645ca926b1
length= 5713956


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

In [ ]:
resp = requests.post(home_url + '/api/sys/firmware_local', data={
    "version_code": 1001,
    "version_name": 'uu_1.0_beta9_1113',
    "firmware_size": 5713956,
    "firmware_name": '/var/111.bin',
})
print(resp.status_code)
print(resp.text)